
# Predição de Sobrevivência de Passageiros do Titanic usando Naive Bayes

Este notebook irá demonstrar como aplicar o algoritmo de Naive Bayes para prever se um passageiro do Titanic sobreviveu ou não, com base em um conjunto de características. 

Além de explicar passo a passo o processo de treinamento do modelo, o notebook também irá salvar o modelo treinado e utilizá-lo para fazer previsões.

## O que é o Algoritmo de Naive Bayes?

Naive Bayes é um algoritmo de classificação baseado no teorema de Bayes, que assume que as características são independentes umas das outras. Ele é chamado de "ingênuo" (naive) porque essa suposição raramente é verdadeira na prática, mas o algoritmo ainda pode funcionar bem em muitos cenários.

Vamos aplicar este algoritmo ao nosso problema para prever se os passageiros sobreviveram ou não ao naufrágio do Titanic.


In [ ]:

# Importando as bibliotecas necessárias
import pandas as pd

# Carregando o dataset
data = pd.read_csv('/mnt/data/train.csv')

# Exibindo as primeiras linhas para entender a estrutura dos dados
data.head()



## Explorando o Dataset

O dataset contém diversas informações sobre os passageiros do Titanic, como:

- **Survived**: Se o passageiro sobreviveu (1) ou não (0).
- **Pclass**: A classe do passageiro (1ª, 2ª, 3ª).
- **Sex**: Gênero do passageiro.
- **Age**: Idade do passageiro.
- **SibSp**: Número de irmãos/cônjuges a bordo.
- **Parch**: Número de pais/filhos a bordo.
- **Fare**: Tarifa paga pelo passageiro.
- **Embarked**: Porto de embarque.

Vamos utilizar essas colunas para treinar nosso modelo de Naive Bayes.


In [ ]:

# Removendo colunas desnecessárias
data_cleaned = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Preenchendo valores ausentes
data_cleaned['Age'].fillna(data_cleaned['Age'].median(), inplace=True)
data_cleaned['Fare'].fillna(data_cleaned['Fare'].median(), inplace=True)
data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0], inplace=True)

# Codificando variáveis categóricas
from sklearn.preprocessing import LabelEncoder
label_encoder_sex = LabelEncoder()
label_encoder_embarked = LabelEncoder()
data_cleaned['Sex'] = label_encoder_sex.fit_transform(data_cleaned['Sex'])
data_cleaned['Embarked'] = label_encoder_embarked.fit_transform(data_cleaned['Embarked'])

# Exibindo os dados limpos
data_cleaned.head()



## Preparação dos Dados

Removemos as colunas irrelevantes para a predição e tratamos os valores ausentes. Além disso, codificamos variáveis categóricas como `Sex` e `Embarked` para que possam ser utilizadas pelo algoritmo de aprendizado de máquina.


In [ ]:

# Separando as features e o target
X = data_cleaned.drop(columns=['Survived'])
y = data_cleaned['Survived']

# Dividindo os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Avaliando a acurácia no conjunto de teste
y_pred = nb_model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy



## Treinamento do Modelo

Após a preparação dos dados, treinamos o modelo de Naive Bayes. A acurácia obtida no conjunto de teste mostra o desempenho do modelo.


In [ ]:

# Salvando o modelo treinado
import joblib
joblib.dump(nb_model, '/mnt/data/naive_bayes_titanic_model.pkl')



## Salvando o Modelo

Aqui salvamos o modelo treinado em um arquivo, para que possamos reutilizá-lo sem precisar treiná-lo novamente.


In [ ]:

# Carregando o modelo salvo e fazendo previsões
loaded_model = joblib.load('/mnt/data/naive_bayes_titanic_model.pkl')

# Função para fazer previsões com o modelo carregado
def predict_survival_with_loaded_model(pclass, sex, age, sibsp, parch, fare, embarked):
    # Codificando as variáveis categóricas
    sex_encoded = label_encoder_sex.transform([sex])[0]
    embarked_encoded = label_encoder_embarked.transform([embarked])[0]
    
    # Criando os dados de entrada no formato esperado pelo modelo
    input_data = [[pclass, sex_encoded, age, sibsp, parch, fare, embarked_encoded]]
    
    # Fazendo a predição com o modelo carregado
    prediction = loaded_model.predict(input_data)[0]
    
    # Retornando o resultado
    return 'Sobreviveu' if prediction == 1 else 'Não sobreviveu'

# Testando a função com o modelo carregado
predict_survival_with_loaded_model(3, 'male', 22, 1, 0, 7.25, 'S')



## Fazendo Previsões com o Modelo Salvo

Agora, podemos carregar o modelo salvo e fazer previsões a partir de novos dados. A função `predict_survival_with_loaded_model` aceita as características de um passageiro e retorna se ele sobreviveu ou não com base no modelo treinado.
